In [6]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [7]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [8]:
#!ls /home/chec/data/bengali

In [9]:
DATA_DIR = '/home/chec/data/bengali'

In [10]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [11]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [12]:
HEIGHT = 137
WIDTH = 236

In [13]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [14]:
#plt.imshow(x)

In [15]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [16]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [17]:
#import pretrainedmodels

In [18]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [19]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [20]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [21]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [22]:
#model.last_linear.in_features

In [23]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [24]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [25]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [26]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [27]:
round(1/9, 6)

0.111111

In [28]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [29]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [30]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [31]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [32]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [33]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [34]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [35]:
np.random.random()

0.3070408134596917

In [36]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam

In [37]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [47]:
args = Namespace()
args.backbone = 'efficientnet-b5'
args.ckp_name = 'model3_efficientnet-b5_fold2_mixup_cutmix.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 1e-4
args.lrs = 'cosine'
args.t_max = 12
args.factor = 0.1
args.patience = 2
args.min_lr = 1e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 512
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [39]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=2)

(200840, 5)
(200840, 32332)
(160678, 5) (40162, 5)


In [48]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


Loaded pretrained weights for efficientnet-b5
model file: ./models/efficientnet-b5/model3_efficientnet-b5_fold2_mixup_cutmix.pth, exist: True
loading ./models/efficientnet-b5/model3_efficientnet-b5_fold2_mixup_cutmix.pth...


In [45]:
train(args)

{'recall': 0.983301, 'recall_grapheme': 0.976311, 'recall_vowel': 0.991086, 'recall_consonant': 0.989495, 'acc_grapheme': 0.975151, 'acc_vowel': 0.99248, 'acc_consonant': 0.990987, 'loss_grapheme': 0.176468, 'loss_vowel': 0.131437, 'loss_consonant': 0.097022}
    0 | 0.000100 | 102400/160678 | 4.1286 | 2.5209 |
val: {'recall': 0.982782, 'recall_grapheme': 0.976237, 'recall_vowel': 0.990603, 'recall_consonant': 0.988052, 'acc_grapheme': 0.9752, 'acc_vowel': 0.992356, 'acc_consonant': 0.990538, 'loss_grapheme': 0.180455, 'loss_vowel': 0.139938, 'loss_consonant': 0.105501}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000098 | 044544/160678 | 3.6575 | 2.5281 |
val: {'recall': 0.982937, 'recall_grapheme': 0.976284, 'recall_vowel': 0.990583, 'recall_consonant': 0.988598, 'acc_grapheme': 0.974877, 'acc_vowel': 0.992281, 'acc_consonant': 0.991061, 'loss_grapheme': 0.188077, 'loss_vowel': 0.140257, 'loss_consonant': 0.105528}
    1 | 0.000093 | 146944/160678 | 1.2643 | 2.4541 |
val: {'recall': 0.981999, 'recall_grapheme': 0.974077, 'recall_vowel': 0.990528, 'recall_consonant': 0.989316, 'acc_grapheme': 0.974777, 'acc_vowel': 0.992381, 'acc_consonant': 0.990713, 'loss_grapheme': 0.175797, 'loss_vowel': 0.136692, 'loss_consonant': 0.099664}
    2 | 0.000086 | 089088/160678 | 3.3413 | 2.2776 |
val: {'recall': 0.982517, 'recall_grapheme': 0.974558, 'recall_vowel': 0.990717, 'recall_consonant': 0.990233, 'acc_grapheme': 0.974877, 'acc_vowel': 0.992456, 'acc_consonant': 0.990912, 'loss_grapheme': 0.171229, 'loss_vowel': 0.118096, 'loss_consonant': 0.092903}
    3 | 0.000075 | 031232/160678 | 3.3601 |

   17 | 0.000086 | 142848/160678 | 3.2847 | 2.2679 |
val: {'recall': 0.984129, 'recall_grapheme': 0.977729, 'recall_vowel': 0.991241, 'recall_consonant': 0.989818, 'acc_grapheme': 0.976993, 'acc_vowel': 0.993053, 'acc_consonant': 0.99136, 'loss_grapheme': 0.153269, 'loss_vowel': 0.111629, 'loss_consonant': 0.087498}
   18 | 0.000075 | 084992/160678 | 2.3258 | 2.1739 |
val: {'recall': 0.983811, 'recall_grapheme': 0.977035, 'recall_vowel': 0.991345, 'recall_consonant': 0.989827, 'acc_grapheme': 0.97642, 'acc_vowel': 0.992904, 'acc_consonant': 0.99141, 'loss_grapheme': 0.149146, 'loss_vowel': 0.106552, 'loss_consonant': 0.083931}
   19 | 0.000063 | 027136/160678 | 4.3916 | 2.3412 |
val: {'recall': 0.984097, 'recall_grapheme': 0.977401, 'recall_vowel': 0.991001, 'recall_consonant': 0.990586, 'acc_grapheme': 0.976918, 'acc_vowel': 0.992879, 'acc_consonant': 0.991659, 'loss_grapheme': 0.153934, 'loss_vowel': 0.111527, 'loss_consonant': 0.091117}
   19 | 0.000051 | 129536/160678 | 1.8129 | 2.

   34 | 0.000063 | 080896/160678 | 3.6998 | 2.2891 |
val: {'recall': 0.985003, 'recall_grapheme': 0.978464, 'recall_vowel': 0.991901, 'recall_consonant': 0.991182, 'acc_grapheme': 0.978014, 'acc_vowel': 0.993178, 'acc_consonant': 0.991709, 'loss_grapheme': 0.16377, 'loss_vowel': 0.125727, 'loss_consonant': 0.095097}
   35 | 0.000051 | 023040/160678 | 2.2858 | 2.0400 |
val: {'recall': 0.985402, 'recall_grapheme': 0.979038, 'recall_vowel': 0.992249, 'recall_consonant': 0.991283, 'acc_grapheme': 0.978512, 'acc_vowel': 0.993427, 'acc_consonant': 0.992057, 'loss_grapheme': 0.154684, 'loss_vowel': 0.11942, 'loss_consonant': 0.091604}
** saved
   35 | 0.000038 | 125440/160678 | 1.1666 | 2.2376 |
val: {'recall': 0.984924, 'recall_grapheme': 0.978415, 'recall_vowel': 0.991613, 'recall_consonant': 0.991253, 'acc_grapheme': 0.978686, 'acc_vowel': 0.993402, 'acc_consonant': 0.991982, 'loss_grapheme': 0.147367, 'loss_vowel': 0.108847, 'loss_consonant': 0.087892}
   36 | 0.000026 | 067584/160678 | 0

...

  100 | 0.000003 | 153600/160678 | 4.0812 | 2.1984 |
val: {'recall': 0.987027, 'recall_grapheme': 0.982086, 'recall_vowel': 0.992296, 'recall_consonant': 0.991642, 'acc_grapheme': 0.981351, 'acc_vowel': 0.99395, 'acc_consonant': 0.993178, 'loss_grapheme': 0.096477, 'loss_vowel': 0.053106, 'loss_consonant': 0.053384}
  101 | 0.000008 | 095744/160678 | 2.4905 | 2.1333 |
val: {'recall': 0.986739, 'recall_grapheme': 0.981932, 'recall_vowel': 0.991962, 'recall_consonant': 0.99113, 'acc_grapheme': 0.981425, 'acc_vowel': 0.993875, 'acc_consonant': 0.993128, 'loss_grapheme': 0.090146, 'loss_vowel': 0.04565, 'loss_consonant': 0.047584}
  102 | 0.000015 | 037888/160678 | 1.9988 | 2.2417 |
val: {'recall': 0.986958, 'recall_grapheme': 0.982412, 'recall_vowel': 0.991738, 'recall_consonant': 0.991271, 'acc_grapheme': 0.981201, 'acc_vowel': 0.993825, 'acc_consonant': 0.993153, 'loss_grapheme': 0.093644, 'loss_vowel': 0.04946, 'loss_consonant': 0.051053}
  102 | 0.000026 | 140288/160678 | 1.7892 | 2.1

  117 | 0.000015 | 091648/160678 | 1.8658 | 2.0752 |
val: {'recall': 0.987171, 'recall_grapheme': 0.982778, 'recall_vowel': 0.992007, 'recall_consonant': 0.991121, 'acc_grapheme': 0.981724, 'acc_vowel': 0.993974, 'acc_consonant': 0.993128, 'loss_grapheme': 0.081713, 'loss_vowel': 0.036795, 'loss_consonant': 0.038026}
  118 | 0.000026 | 033792/160678 | 2.0477 | 2.1255 |
val: {'recall': 0.987485, 'recall_grapheme': 0.983092, 'recall_vowel': 0.992382, 'recall_consonant': 0.991372, 'acc_grapheme': 0.981948, 'acc_vowel': 0.993974, 'acc_consonant': 0.993551, 'loss_grapheme': 0.082686, 'loss_vowel': 0.038439, 'loss_consonant': 0.039308}
  118 | 0.000038 | 136192/160678 | 1.8875 | 2.1460 |
val: {'recall': 0.987293, 'recall_grapheme': 0.982672, 'recall_vowel': 0.992106, 'recall_consonant': 0.991722, 'acc_grapheme': 0.981724, 'acc_vowel': 0.993925, 'acc_consonant': 0.993252, 'loss_grapheme': 0.087775, 'loss_vowel': 0.04228, 'loss_consonant': 0.042987}
  119 | 0.000050 | 078336/160678 | 3.0339 | 

  134 | 0.000038 | 029696/160678 | 4.0381 | 2.1550 |
val: {'recall': 0.988071, 'recall_grapheme': 0.983689, 'recall_vowel': 0.992547, 'recall_consonant': 0.992359, 'acc_grapheme': 0.982546, 'acc_vowel': 0.994099, 'acc_consonant': 0.993576, 'loss_grapheme': 0.078805, 'loss_vowel': 0.036621, 'loss_consonant': 0.035895}
  134 | 0.000051 | 132096/160678 | 4.1580 | 2.0846 |
val: {'recall': 0.987577, 'recall_grapheme': 0.98293, 'recall_vowel': 0.992483, 'recall_consonant': 0.991964, 'acc_grapheme': 0.98277, 'acc_vowel': 0.994223, 'acc_consonant': 0.993576, 'loss_grapheme': 0.07869, 'loss_vowel': 0.03665, 'loss_consonant': 0.036304}
  135 | 0.000063 | 074240/160678 | 1.9431 | 2.0436 |
val: {'recall': 0.987482, 'recall_grapheme': 0.983351, 'recall_vowel': 0.99225, 'recall_consonant': 0.990975, 'acc_grapheme': 0.982371, 'acc_vowel': 0.993974, 'acc_consonant': 0.993302, 'loss_grapheme': 0.078803, 'loss_vowel': 0.036645, 'loss_consonant': 0.034531}
  136 | 0.000075 | 016384/160678 | 2.1622 | 2.24

  150 | 0.000063 | 128000/160678 | 0.8950 | 2.1408 |
val: {'recall': 0.988174, 'recall_grapheme': 0.984081, 'recall_vowel': 0.992518, 'recall_consonant': 0.992017, 'acc_grapheme': 0.982795, 'acc_vowel': 0.994149, 'acc_consonant': 0.9939, 'loss_grapheme': 0.076678, 'loss_vowel': 0.035728, 'loss_consonant': 0.032817}
** saved
  151 | 0.000075 | 070144/160678 | 1.5744 | 2.2191 |
val: {'recall': 0.98813, 'recall_grapheme': 0.983324, 'recall_vowel': 0.992771, 'recall_consonant': 0.993102, 'acc_grapheme': 0.982496, 'acc_vowel': 0.994273, 'acc_consonant': 0.993925, 'loss_grapheme': 0.07754, 'loss_vowel': 0.034452, 'loss_consonant': 0.032111}
  152 | 0.000086 | 012288/160678 | 0.3801 | 2.0123 |
val: {'recall': 0.98752, 'recall_grapheme': 0.982945, 'recall_vowel': 0.99299, 'recall_consonant': 0.9912, 'acc_grapheme': 0.982247, 'acc_vowel': 0.994398, 'acc_consonant': 0.993825, 'loss_grapheme': 0.07706, 'loss_vowel': 0.035688, 'loss_consonant': 0.0327}
  152 | 0.000093 | 114688/160678 | 1.8573 | 2

KeyboardInterrupt: 

In [41]:
train(args)

{'recall': 0.055809, 'recall_grapheme': 0.002094, 'recall_vowel': 0.082583, 'recall_consonant': 0.136466, 'acc_grapheme': 0.004756, 'acc_vowel': 0.062123, 'acc_consonant': 0.30372, 'loss_grapheme': 5.147646, 'loss_vowel': 2.432934, 'loss_consonant': 1.875116}


/home/chec/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


    0 | 0.001000 | 102400/160678 | 4.3291 | 5.1119 |
val: {'recall': 0.921762, 'recall_grapheme': 0.88298, 'recall_vowel': 0.956336, 'recall_consonant': 0.964751, 'acc_grapheme': 0.860615, 'acc_vowel': 0.940367, 'acc_consonant': 0.948284, 'loss_grapheme': 0.89797, 'loss_vowel': 0.407851, 'loss_consonant': 0.264921}
** saved
    1 | 0.001000 | 044544/160678 | 3.7370 | 3.7624 |
val: {'recall': 0.94627, 'recall_grapheme': 0.920104, 'recall_vowel': 0.971516, 'recall_consonant': 0.973355, 'acc_grapheme': 0.910637, 'acc_vowel': 0.968154, 'acc_consonant': 0.964071, 'loss_grapheme': 0.673879, 'loss_vowel': 0.235241, 'loss_consonant': 0.180345}
** saved
    1 | 0.001000 | 146944/160678 | 3.8561 | 3.5787 |
val: {'recall': 0.957307, 'recall_grapheme': 0.93805, 'recall_vowel': 0.973764, 'recall_consonant': 0.979362, 'acc_grapheme': 0.926871, 'acc_vowel': 0.965739, 'acc_consonant': 0.968129, 'loss_grapheme': 0.570661, 'loss_vowel': 0.316512, 'loss_consonant': 0.214272}
** saved
    2 | 0.001000 | 0

** saved
   17 | 0.000020 | 040448/160678 | 1.3836 | 2.3352 |
val: {'recall': 0.982445, 'recall_grapheme': 0.974879, 'recall_vowel': 0.990231, 'recall_consonant': 0.989792, 'acc_grapheme': 0.974155, 'acc_vowel': 0.992107, 'acc_consonant': 0.990812, 'loss_grapheme': 0.182533, 'loss_vowel': 0.13802, 'loss_consonant': 0.102619}
   17 | 0.000020 | 142848/160678 | 1.9077 | 2.4087 |
val: {'recall': 0.982927, 'recall_grapheme': 0.9754, 'recall_vowel': 0.990945, 'recall_consonant': 0.989961, 'acc_grapheme': 0.974354, 'acc_vowel': 0.992256, 'acc_consonant': 0.990787, 'loss_grapheme': 0.180132, 'loss_vowel': 0.132313, 'loss_consonant': 0.098958}
** saved
   18 | 0.000020 | 084992/160678 | 1.2952 | 2.2707 |
val: {'recall': 0.982633, 'recall_grapheme': 0.975842, 'recall_vowel': 0.990668, 'recall_consonant': 0.98818, 'acc_grapheme': 0.974852, 'acc_vowel': 0.992082, 'acc_consonant': 0.990812, 'loss_grapheme': 0.179789, 'loss_vowel': 0.129625, 'loss_consonant': 0.097027}
   19 | 0.000020 | 027136/160

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [46]:
del model